In [16]:
from __future__ import division
import numpy as np
import os, sys
import matplotlib
#matplotlib.use('Agg')
%matplotlib nbagg
import matplotlib.pyplot as plt
import pandas as pd
import subprocess
from collections import defaultdict, Counter


def getCDF(data):
    xdata = np.sort(data)
    ydata = [i/len(xdata) for i in range(len(xdata))]
    return xdata, ydata

# STEPS
- load pcap -> tshark extract
- load tshark csv
- filter to a record only
- key srcip
- apply feature functions
 - number of dst
 - number of A queries
 - number of unique A queries
 - diff all: avg, 50%, min, max
 - most popular query 1,2,3
 - most popular query diff 1,2,3: avg
 - least popular query
 - FREQUENCY BASED FEATURES (need to finish resampling code #todo)

In [2]:
# load from data/ and extract to processed/
infile='data/split_6hour_00000_20170206235900.pcap'
outfile='processed/split_6hour_00000_20170206235900.csv'

cmd_extract = 'tshark -r '+infile+' -E separator="|" -T fields -e frame.time_epoch \
-e frame.time_relative -e ip.src -e ip.dst -e dns.qry.name -e dns.qry.type -Y "dns.flags.response eq 0" \
> '+outfile

# ONLY DO THIS FOR NEW PCAP FILES
# subprocess.check_output(cmd_extract, shell=True)

In [3]:
# load csv from processed
df = pd.read_csv("processed/split_hour_00000_20170206235900.csv", sep="|", error_bad_lines=False,
                names=['time', 'time_relative', 'srcip', 'dstip', 'dnsqry', 'dnstype'],
                #dtype={'time': pd.np.float, 'time_relative': pd.np.float64, 'dnstype': pd.np.int},
                ).dropna()

/usr/local/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (0,1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
df['time'] = df['time'].astype(float)
df['time_relative'] = df['time_relative'].astype(float)

### Stats by dns.qry.type
- convert qry type hex -> int
- A record == 1
- for 2 or more qry type take the first one after splitting

In [5]:
dnstype_count = df.groupby('dnstype')['dnsqry'].count()
dnstype_count.sort_values(inplace=True, ascending=False)

print "Percentage of A Records = ", dnstype_count.iloc[0]/dnstype_count.sum()*100

dnstype_count.head(15)

Percentage of A Records =  77.8158606734


dnstype
0x00000001               1505933
0x0000001c                196059
0x0000000c                153732
0x0000000f                 37187
0x00000021                 33905
0x00000010                  3572
0x00000006                  3331
0x0000002b                   500
0x000000ff                   319
0x000000f9                   185
0x00000001,0x0000001c        147
0x00000002                   110
0x0000b71f                    36
0x0000a21f                    14
0x0000a11f                    11
Name: dnsqry, dtype: int64

In [6]:
# convert dnstype to int (take only first type in case of 2)
df['dnstype'] = df['dnstype'].apply(lambda x: int( x.split(',')[0], 16))

dnstype_count = df.groupby('dnstype')['dnsqry'].count()
dnstype_count.sort_values(inplace=True, ascending=False)

print "New percentage of A Records = ", dnstype_count.iloc[0]/dnstype_count.sum()*100

New percentage of A Records =  77.8234565834


In [34]:
df.head()

,time,time_relative,srcip,dstip,dnsqry,dnstype
0,1.486444e+09,0.000567,10.9.151.75,10.8.0.5,safebrowsing.google.com,1
1,1.486444e+09,0.000744,140.180.223.155,128.112.129.209,51.255.61.103.sbl-xbl.spamhaus.dnsbl,1
2,1.486444e+09,0.000853,140.180.223.155,128.112.129.209,51.255.61.103.sbl-xbl.spamhaus.dnsbl,1
3,1.486444e+09,0.003419,10.9.38.65,10.8.0.5,tpc.googlesyndication.com,1
4,1.486444e+09,0.003675,10.6.63.107,128.112.129.209,98.63.6.10.in-addr.arpa,12


### filter to A records only

In [7]:
df_a = df[df.dnstype == 1]

## group by srcip as KEY and extract features

In [13]:
df_sample = df_a.head(5000).copy()

In [14]:
grouped = df_sample.groupby('srcip')

In [71]:
# grouped.agg({'C' : np.sum, 'D' : lambda x: np.std(x, ddof=1)})
data = defaultdict(list)

for srcip, group in grouped:
    # key
    data['srcip'].append( srcip )
    
    # DSTIP
    # number of unique dstip
    data['dstip.count'].append( len( set( group['dstip'] ) ) )
    
    # most common dstip
    data['dstip.most_popular'].append( Counter( group['dstip'] ).most_common(1)[0][0] )
    
    # DNS QRY
    # count of total domains
    data['dnsqry.count'].append( group['dnsqry'].count() )
    
    # count of unique domains
    data['dnsqry.unique.count'].append( len( set( group['dnsqry'] )) )
    
    # TIME DIFF ALL - DESCRIBE
    td = group['time'].diff()  #fold(1)
    data['time_diff.mean'].append( td.mean() )
    data['time_diff.std'].append( td.std() )
    data['time_diff.min'].append( td.min() ) 
    data['time_diff.max'].append( td.max() )
    data['time_diff.median'].append( td.median() )
    
    # TOP 3 most popular domains and their time diffs
    top3 = Counter( group['dnsqry'] ).most_common(3)
    for num in range(3):
        column_name = 'dnsqry.most_pop'+ str(num)
        
        if len(top3)>num:
            pop_domains = top3[num]
            domain = pop_domains[0]  #domain
            occurance = pop_domains[1]   #number of lookups
            data[column_name + '.domain'].append( domain )   
            data[column_name + '.count'].append( occurance )
            # time diff
            td = group[ group['dnsqry']==domain ]['time'].diff()
            data[column_name + '.time_diff.mean'].append( td.mean() )
            data[column_name + '.time_diff.std'].append( td.std() )
            data[column_name + '.time_diff.min'].append( td.min() )
            data[column_name + '.time_diff.max'].append( td.max() )
            data[column_name + '.time_diff.median'].append( td.median() )
            
        else:
            data[column_name + '.domain'].append( np.nan )   
            data[column_name + '.count'].append( np.nan )
            # time diff
            data[column_name + '.time_diff.mean'].append( np.nan )
            data[column_name + '.time_diff.std'].append( np.nan )
            data[column_name + '.time_diff.min'].append( np.nan )
            data[column_name + '.time_diff.max'].append( np.nan )
            data[column_name + '.time_diff.median'].append( np.nan )
            
    # Least popular domain
    bottom = Counter( group['dnsqry'] ).most_common()[-1]
    data['dnsqry.least_pop.domain'].append( bottom[0] )    #domain
    data['dnsqry.least_pop.count'].append( bottom[1] )     #occurances

    # FREQUENCY BASED FEATURES
    #TODO

In [72]:
# check num entries before converting to dataframe
for k,v in data.items():
    print len(v), "\t", k

1162 	dstip.most_popular
1162 	dnsqry.most_pop0.time_diff.median
1162 	dnsqry.count
1162 	dnsqry.most_pop0.count
1162 	dnsqry.most_pop0.domain
1162 	time_diff.median
1162 	dnsqry.most_pop0.time_diff.max
1162 	dstip.count
1162 	dnsqry.least_pop.count
1162 	dnsqry.most_pop2.time_diff.min
1162 	dnsqry.most_pop1.time_diff.std
1162 	dnsqry.most_pop1.time_diff.median
1162 	dnsqry.most_pop2.time_diff.median
1162 	dnsqry.most_pop2.count
1162 	dnsqry.most_pop1.time_diff.min
1162 	time_diff.std
1162 	dnsqry.most_pop0.time_diff.mean
1162 	dnsqry.most_pop1.time_diff.mean
1162 	dnsqry.least_pop.domain
1162 	dnsqry.most_pop2.time_diff.std
1162 	dnsqry.most_pop1.time_diff.max
1162 	srcip
1162 	dnsqry.most_pop1.domain
1162 	dnsqry.unique.count
1162 	time_diff.max
1162 	dnsqry.most_pop1.count
1162 	dnsqry.most_pop2.time_diff.max
1162 	time_diff.mean
1162 	dnsqry.most_pop2.domain
1162 	time_diff.min
1162 	dnsqry.most_pop0.time_diff.min
1162 	dnsqry.most_pop2.time_diff.mean
1162 	dnsqry.most_pop0.time_di

In [73]:
df_features = pd.DataFrame(data)

In [74]:
df_features

,dnsqry.count,dnsqry.least_pop.count,dnsqry.least_pop.domain,dnsqry.most_pop0.count,dnsqry.most_pop0.domain,dnsqry.most_pop0.time_diff.max,dnsqry.most_pop0.time_diff.mean,dnsqry.most_pop0.time_diff.median,dnsqry.most_pop0.time_diff.min,dnsqry.most_pop0.time_diff.std,...,dnsqry.most_pop2.time_diff.std,dnsqry.unique.count,dstip.count,dstip.most_popular,srcip,time_diff.max,time_diff.mean,time_diff.median,time_diff.min,time_diff.std
0,5,1,devtool.Princeton.EDU,1,fac221w-acad.Princeton.EDU,NaN,NaN,NaN,NaN,NaN,...,NaN,5,1,128.112.129.7,10.6.48.12,4.975024,2.008973,1.529118,0.002633,2.147026
1,1,1,clients1.google.com,1,clients1.google.com,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,128.112.128.1,10.6.48.13,NaN,NaN,NaN,NaN,NaN
2,20,1,PHY-207.princeton.edu,2,INR-GLDK6V1.princeton.edu,3.511184,3.511184,3.511184,3.511184,NaN,...,NaN,19,1,128.112.128.1,10.6.48.172,3.999641,0.426321,0.006884,0.000201,1.174826
3,1,1,endpoint.ingress.rapid7.com,1,endpoint.ingress.rapid7.com,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,128.112.128.1,10.6.48.20,NaN,NaN,NaN,NaN,NaN
4,1,1,endpoint.ingress.rapid7.com,1,endpoint.ingress.rapid7.com,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,128.112.128.1,10.6.48.22,NaN,NaN,NaN,NaN,NaN
5,1,1,PDOM08.pu.win.princeton.edu,1,PDOM08.pu.win.princeton.edu,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,128.112.128.1,10.6.48.48,NaN,NaN,NaN,NaN,NaN
6,24,1,essesx211v.princeton.edu,2,essesx201v.princeton.edu,10.000525,10.000525,10.000525,10.000525,NaN,...,NaN,18,1,128.112.128.1,10.6.48.80,2.051464,0.473554,0.304159,0.014711,0.442319
7,1,1,clients4.google.com,1,clients4.google.com,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,128.112.128.1,10.6.49.1,NaN,NaN,NaN,NaN,NaN
8,1,1,TSM8.PRINCETON.EDU,1,TSM8.PRINCETON.EDU,NaN,NaN,NaN,NaN,NaN,...,NaN,1,1,128.112.128.1,10.6.49.115,NaN,NaN,NaN,NaN,NaN
9,2,1,PDOM05.pu.win.princeton.edu,1,PDOM08.pu.win.princeton.edu,NaN,NaN,NaN,NaN,NaN,...,NaN,2,1,128.112.128.1,10.6.49.141,0.022039,0.022039,0.022039,0.022039,NaN
